In [1]:
%pdb

Automatic pdb calling has been turned ON


In [1]:
from sleepmate.agent import *

In [3]:
!rm -f /Users/cck197/Downloads/virtual_greg/memory.pkl

In [2]:
from sleepmate.helpful_scripts import *

In [7]:
from sleepmate.diary import *
from sleepmate.goal import *

In [9]:
diary_probe()

True

In [6]:
(start, end) = (parse_date("2023-10-01"), datetime.now())

In [7]:
start, end

(datetime.datetime(2023, 10, 1, 0, 0),
 datetime.datetime(2023, 10, 19, 18, 5, 12, 539620))

In [14]:
goal_refused("diary_probe", start, end)

<GoalRefusal: GoalRefusal object>

In [10]:
diary_entry()

True

In [9]:
def_no_sleep_diary()

False

In [8]:
DBGoalRefusal.objects.delete()

2

In [13]:
[o.goal for o in DBGoalRefusal.objects()]

['diary_probe', 'daily_routine']

In [ ]:
DBSleepDiaryEntry.objects(user=get_current_user()).count()

In [11]:
from sleepmate.goal import DBGoalRefusal

In [3]:
imported = import_attrs(["GOALS", "GOAL_HANDLERS"])

In [3]:
imported = import_attrs(["TOOLS"])

In [33]:
memory = X.load_memory()

X.load_memory filename='/Users/cck197/Downloads/virtual_greg/memory.pkl'


In [34]:
memory.chat_memory.messages#[:-6]

[AIMessage(content="what's your name?"),
 HumanMessage(content='Chris'),
 AIMessage(content="what's your email?"),
 HumanMessage(content='chris@nourishbalancethrive.com'),
 HumanMessage(content='hey'),
 AIMessage(content="Hello, Chris! I hope you're doing well. Is now a good time to record a sleep diary entry?"),
 HumanMessage(content='sure'),
 AIMessage(content="Fantastic! Let's start with the date of the entry. By default, we'll use yesterday's date. Is that okay, or would you like to specify a different date?"),
 HumanMessage(content='yesterday is fine'),
 AIMessage(content="Great, we'll use yesterday's date. Now, could you please tell me what time you went to bed?"),
 HumanMessage(content='21:15'),
 AIMessage(content='Thank you, Chris. And at what time did you try to fall asleep?'),
 HumanMessage(content='21:15'),
 AIMessage(content="Alright, you tried to fall asleep at the same time you went to bed. That's quite efficient! Now, how long did it take you to fall asleep, in minutes?"

In [35]:
memory.chat_memory.messages = memory.chat_memory.messages[:-2]

In [36]:
X.save_memory(memory)